In [2]:

from tqdm import tqdm
import re
import pickle
import json
import torch
from langchain.embeddings import SentenceTransformerEmbeddings
from transformers import AutoModel, AutoTokenizer
from langchain.vectorstores import Chroma
import json
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from transformers import AdamW
import numpy as np
import random
from IPython.display import HTML, display
import getpass
import logging
import os
import torch
import yaml
from ludwig.api import LudwigModel
from datasets import load_dataset
import numpy as np; np.random.seed(123)
import pandas as pd
from tqdm import tqdm
from IPython.display import HTML, display
from transformers import AutoModelForSeq2SeqLM, AutoModelForCausalLM
from transformers import AutoTokenizer
from transformers import GenerationConfig
import evaluate
import json

/home/synthesisproject/anaconda3/lib/python3.8/site-packages/dask/dataframe/utils.py:369: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)
/home/synthesisproject/anaconda3/lib/python3.8/site-packages/dask/dataframe/utils.py:369: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)
/home/synthesisproject/anaconda3/lib/python3.8/site-packages/dask/dataframe/utils.py:369: FutureWarning: pandas.UInt64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)


In [17]:
torch.__version__

'2.0.1+cu117'

In [4]:
gt= pd.read_csv('Table dataset.csv')
gt.head()

,Prompt,Winner
0,"Oxides weight percentage content in EAF slag, ...","Material Name, SiO2, Al2O3, Fe2O3, CaO, MgO, S..."
1,Properties of cement and fly ashes used \n [['...,"Material Name,SiO2,Al2O3,Fe2O3,CaO,MgO,SO3,TiO..."
2,Chemical composition of SFS. \n [['Constituent...,"Material Name,SiO2,Al2O3,Fe2O3,CaO,MgO,SO3,TiO..."
3,"Chemical compositions of cement, silica fume a...","Material Name,SiO2,Al2O3,Fe2O3,CaO,MgO,SO3,TiO..."
4,"Cement composition oxide wt.% \n [['SiO2', 'Al...","Material Name,SiO2,Al2O3,Fe2O3,CaO,MgO,SO3,TiO..."


In [5]:
dataset_c = []
for _, (Table, Winner) in gt.iterrows():
    Prompt = f"""<s>[INST]Primary Task:Parse an unstructured HTML table and reformat it into a CSV file.
    CSV Format Requirements: Ensure the CSV includes columns for Material Name, SiO2, Al2O3, Fe2O3, CaO, MgO, SO3, TiO2, MnO, K2O, Na2O, LOI.
    Handling Missing Data: Use 'NaN' for any missing data in these columns.
    Non-relevant Data: If the table's content is not related to the specified chemicals, respond with 'NR' only.
    
  
    Unstructured HTML table: {Table}
    [INST]"""


    dataset_c.append({"Prompt": Prompt, "Winner": Winner})

In [6]:
Dataset = pd.DataFrame(dataset_c)
Dataset.head()

,Prompt,Winner
0,<s>[INST]Primary Task:Parse an unstructured HT...,"Material Name, SiO2, Al2O3, Fe2O3, CaO, MgO, S..."
1,<s>[INST]Primary Task:Parse an unstructured HT...,"Material Name,SiO2,Al2O3,Fe2O3,CaO,MgO,SO3,TiO..."
2,<s>[INST]Primary Task:Parse an unstructured HT...,"Material Name,SiO2,Al2O3,Fe2O3,CaO,MgO,SO3,TiO..."
3,<s>[INST]Primary Task:Parse an unstructured HT...,"Material Name,SiO2,Al2O3,Fe2O3,CaO,MgO,SO3,TiO..."
4,<s>[INST]Primary Task:Parse an unstructured HT...,"Material Name,SiO2,Al2O3,Fe2O3,CaO,MgO,SO3,TiO..."


In [7]:
dataset_c[1]

{'Prompt': "<s>[INST]Primary Task:Parse an unstructured HTML table and reformat it into a CSV file.\n    CSV Format Requirements: Ensure the CSV includes columns for Material Name, SiO2, Al2O3, Fe2O3, CaO, MgO, SO3, TiO2, MnO, K2O, Na2O, LOI.\n    Handling Missing Data: Use 'NaN' for any missing data in these columns.\n    Non-relevant Data: If the table's content is not related to the specified chemicals, respond with 'NR' only.\n    \n  \n    Unstructured HTML table: Properties of cement and fly ashes used \n [['Chemical composition', 'Chemical composition', 'Chemical composition', 'Chemical composition', 'Chemical composition', ''], ['Oxide (%)', 'PC', 'HL1', 'HL2', 'LL1', 'LL2'], ['CaO', '63.3', '20.3', '14.0', '4.4', '2.0'], ['SiO2', '20.2', '43.3', '47.1', '58.6', '59.4'], ['Al2O3', '5.9', '25.1', '17.4', '21.9', '25.8'], ['Fe2O3', '3.3', '4.9', '8.3', '9.3', '5.8'], ['MgO', '1.8', '0.8', '1.9', '1.4', '0.6'], ['SO3', '2.9', '2.5', '4.6', '0.4', '0.1'], ['K2O', '0.8', '1.3', '1.8

In [8]:
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(dataset_c, test_size=0.13)
train_df= pd.DataFrame(train_data)
test_df= pd.DataFrame(test_data)

In [9]:

train_df.head()

,Prompt,Winner
0,<s>[INST]Primary Task:Parse an unstructured HT...,"Material Name,SiO2,Al2O3,Fe2O3,CaO,MgO,SO3,TiO..."
1,<s>[INST]Primary Task:Parse an unstructured HT...,"Material Name, SiO2, Al2O3, Fe2O3, CaO, MgO, S..."
2,<s>[INST]Primary Task:Parse an unstructured HT...,NR
3,<s>[INST]Primary Task:Parse an unstructured HT...,"""Material Name,SiO2,Al2O3,Fe2O3,CaO,MgO,SO3,Ti..."
4,<s>[INST]Primary Task:Parse an unstructured HT...,"Material Name, SiO2, Al2O3, Fe2O3, CaO, MgO, S..."


In [10]:
train_df.loc[3]['Winner']

'"Material Name,SiO2,Al2O3,Fe2O3,CaO,MgO,SO3,TiO2,MnO,K2O,Na2O,LOI\nPortland Cement,19.701,4.653,3.482,61.123,2.059,3.007,0.271,NaN,0.695,0.154,NaN"'

In [11]:
train_df.loc[3]['Prompt']

"<s>[INST]Primary Task:Parse an unstructured HTML table and reformat it into a CSV file.\n    CSV Format Requirements: Ensure the CSV includes columns for Material Name, SiO2, Al2O3, Fe2O3, CaO, MgO, SO3, TiO2, MnO, K2O, Na2O, LOI.\n    Handling Missing Data: Use 'NaN' for any missing data in these columns.\n    Non-relevant Data: If the table's content is not related to the specified chemicals, respond with 'NR' only.\n    \n  \n    Unstructured HTML table: Chemical and physical properties of LSP. \n [['Materials', 'Chemical composition (%)', 'Chemical composition (%)', 'Chemical composition (%)', 'Typical properties', 'Typical properties'], ['', 'CaCO3', 'MgCO3', 'Fe2O3', 'Specific gravity', 'Bulk density (gr/cm3)'], ['Limestone Powder', '96', '1', '<0.1', '2.65', '2.3']]\n    [INST]"

In [12]:
test_df.head()

,Prompt,Winner
0,<s>[INST]Primary Task:Parse an unstructured HT...,"Material Name,SiO2,Al2O3,Fe2O3,CaO,MgO,SO3,TiO..."
1,<s>[INST]Primary Task:Parse an unstructured HT...,"Material Name,SiO2,Al2O3,Fe2O3,CaO,MgO,SO3,TiO..."
2,<s>[INST]Primary Task:Parse an unstructured HT...,NR
3,<s>[INST]Primary Task:Parse an unstructured HT...,NR
4,<s>[INST]Primary Task:Parse an unstructured HT...,"""Material Name,SiO2,Al2O3,Fe2O3,CaO,MgO,SO3,Ti..."


In [13]:
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))

get_ipython().events.register('pre_run_cell', set_css)

def clear_cache():
  if torch.cuda.is_available():
    model = None
    torch.cuda.empty_cache()

In [ ]:
model = None
clear_cache()


qlora_fine_tuning_config = yaml.safe_load(
"""
  model_type: llm
  base_model:   mistralai/Mistral-7B-v0.1


  input_features:
    - name: Prompt
      type: text


  output_features:
    - name: Winner
      type: text


  prompt:
    template: >-

      ### Prompt: {Prompt}

      ### Winner :
  
  quantization:
    bits: 8

  generation:
    temperature: 0.1
    max_new_tokens: 2048
    
  preprocessing:
    split:
       probabilities:
        - 0.9
        - 0.05
        - 0.05
    global_max_sequence_length: 2048
    
  adapter:
    type: lora

  trainer:
    type: finetune
    epochs: 3
    batch_size: 1
    eval_batch_size: 1
    enable_gradient_checkpointing: true
    gradient_accumulation_steps: 16
    learning_rate: 0.0002
    optimizer:
      type: adamw
      params:
        eps: 1.e-8
        betas:
          - 0.9
          - 0.999
        weight_decay: 0
    learning_rate_scheduler:
      warmup_fraction: 0.03
      reduce_on_plateau: 0
  """
  )


new_model = LudwigModel(config=qlora_fine_tuning_config, logging_level=logging.INFO)
results = new_model.train(dataset=Dataset)

In [ ]:
save_path='QLORA_mistral_TE_CC'
new_model.save(save_path)

In [ ]:
predictions, _ = new_model.predict(dataset=test_df)


In [ ]:
results = predictions.Winner_response
print(results)